In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from mc2.data_management import DATA_ROOT, CACHE_ROOT, load_data_into_pandas_df
from mc2.features.features import add_fe
from mc2.features.features_torch import Featurizer

In [ ]:
mat_lbl = "3C90"
data_d = load_data_into_pandas_df(material=mat_lbl)
for k in data_d.keys():
    print(k)
#for i in range(1, 6):
#    print(data_d[f"{mat_lbl}_{i}_B"].shape)

# Some visualizations with a certain frequency

In [ ]:
no = 1
b, h, t = data_d[f"{mat_lbl}_{no}_B"].to_numpy(), data_d[f"{mat_lbl}_{no}_H"].to_numpy(), data_d[f"{mat_lbl}_{no}_T"].to_numpy()
b *= 1e3  # in mT

In [ ]:
# random ts next to each other
fig, axes = plt.subplots(5, 2, figsize=(9, 5), sharex=True, sharey="col")

choice = np.random.choice(len(b), 5)
for i, c in enumerate(choice):
    axes[i, 0].plot(b[c, :].T)
    axes[i, 1].plot(h[c, :].T, color='tab:orange')
    axes[i, 1].set_ylim(-400, 400)
for ax in axes.flatten():
    ax.grid(alpha=0.5)
axes[0, 0].set_title("B in mT")
axes[0, 1].set_title("H in A/m")
for ax in axes[-1, :]:
    ax.set_xlabel("Step")
fig.suptitle(mat_lbl)

In [ ]:
# random ts on top of each other (scaled)
fig, axes = plt.subplots(5, 1, figsize=(9, 9), sharex=True, sharey="col")

choice = np.random.choice(len(b), 5)
for i, c in enumerate(choice):
    axes[i].plot(b[c, :].T, label='B in mT')
    axes[i].plot(h[c, :].T, color='tab:orange', label="H in A/m")
    #axes[i].set_ylim(-500, 500)
for ax in axes.flatten():
    ax.grid(alpha=0.5)

ax.legend()

ax.set_xlabel("Step")
fig.suptitle(mat_lbl)

In [ ]:
# H over B
fig, axes = plt.subplots(1, 5, figsize=(12, 4), sharex=True, sharey="row")

choice = np.random.choice(len(b), 5)
for i, (c, ax) in enumerate(zip(choice, axes.flatten())):
    ax.plot(b[c, :].T, h[c, :].T, lw=0.5)
    #ax.set_ylim(-250, 250)
    if i == 0:
        ax.set_ylabel("H in A/m")
    ax.set_xlabel("B in mT")
    
for ax in axes.flatten():
    ax.grid(alpha=0.5)


fig.suptitle(mat_lbl)

In [ ]:
# H over B, random trajectories plotted on top of each other

choice = np.random.choice(len(b), 5)
fig = plt.figure()
ax = fig.gca()
for i, c in enumerate(choice):
    ax.plot(b[c, :].T, h[c, :].T, lw=0.5)

ax.set_ylabel("H in A/m")
ax.set_xlabel("B in mT")
#ax.set_ylim(-500, 500)
ax.grid(alpha=0.5)


fig.suptitle(mat_lbl)

In [ ]:
plt.hist(b.ravel(), label="B", bins=100, alpha=0.7)
plt.hist(h.ravel(), label="H", bins=100, alpha=.7)
plt.grid(alpha=0.3)
plt.legend()

In [ ]:
# visualize FE
featurizer = Featurizer()
choice = np.random.choice(len(b), 5)
fig, axes= plt.subplots(5, 1, figsize=(12, 12), sharex=True, sharey=False)

T_lim = 500
b_selected = b[choice, :]
h_selected = h[choice, :]

b_selected_MND = np.dstack([f.numpy() for f in featurizer.add_fe(torch.from_numpy(b_selected))])
titles = ["B", "dB/dt", "d²B/dt²", "smoothed B", "PWM of B"]
for i, ax in enumerate(axes.flatten()):
    ax.plot(b_selected_MND[:1, :T_lim, i].T, lw=0.5)
    ax.set_title(titles[i])
    ax.grid(alpha=0.5)
ax.plot(h_selected[:1, :T_lim].T / 100)
#ax.set_ylabel("H in A/m")
ax.set_ylabel("B in mT")
#ax.set_ylim(-500, 500)



fig.suptitle(mat_lbl)

## Play around with transforms

In [ ]:
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(13, 5))

for i, c in enumerate(choice):
    ax_i = int(t[c].item() / 25 - 1)
    ax = axes[ax_i]
    b_selected = b[c, :].T
    h_selected = h[c, :].T 
    h_transformed = np.tanh(2* h_selected / np.abs(h).max())
    #ax.plot(b_single, h_single, lw=0.5)
    ax.plot(b_selected, h_transformed, lw=0.5)

for ax, lbl in zip(axes, [25, 50, 75]):
    ax.set_ylabel("H in A/m")
    ax.set_xlabel("B in mT")
    #ax.set_ylim(-500, 500)
    ax.grid(alpha=0.5)
    ax.set_title(f"{lbl} °C")


fig.suptitle(mat_lbl)
fig.tight_layout()

In [ ]:
plt.plot(h_transformed)
plt.plot(h_selected / np.abs(h).max())

# Investigate concave hulls

In [ ]:
# show all BH curves across frequency and temperature


temps = np.arange(25, 100, 25)  # in °C
n_freqs = max([int(k.split("_")[1]) for k in data_d.keys()])

fig, axes = plt.subplots(n_freqs, temps.size, sharex=True, sharey=True, figsize=(13, 10))
for i_temp, temp in enumerate(temps.tolist()):
    for i_freq in range(n_freqs):
        ax = axes[i_freq, i_temp]
        key = f"{mat_lbl}_{i_freq+1}_B"
        b_all = data_d[key].to_numpy()
        h_all = data_d[key.replace("_B", "_H")].to_numpy()
        t_all = data_d[key.replace("_B", "_T")].to_numpy().ravel()
        sel = t_all == temp

        b_selected = b_all[sel, :].T
        h_selected = h_all[sel, :].T 
        
        ax.plot(b_selected, h_selected, lw=0.5)
        if i_temp == 0:
            ax.set_ylabel("H in A/m")
        if i_freq == n_freqs - 1:
            ax.set_xlabel("B in mT")
        if i_freq == 0:
            ax.set_title(f"{temp} °C")
        ax.grid(alpha=0.5)

In [ ]:
# show those at first frequency and 25°C more in-depth
fig, ax = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(10, 5))
selected_temp = 25  # in °C
key = f"{mat_lbl}_1_B"
b_all = data_d[key].to_numpy()
h_all = data_d[key.replace("_B", "_H")].to_numpy()
t_all = data_d[key.replace("_B", "_T")].to_numpy().ravel()
sel = t_all == selected_temp

b_selected = b_all[sel, :].T
h_selected = h_all[sel, :].T 

ax.plot(b_selected, h_selected, lw=0.5)
ax.set_ylabel("H in A/m")
ax.set_xlabel("B in mT")
ax.set_title(f"{selected_temp} °C")
ax.grid(alpha=0.5)

In [ ]:
if False:
    # dont run this code, long-running
    from concave_hull import concave_hull_indexes

    bh_col_stack = np.column_stack((b_selected.ravel(), h_selected.ravel()))

    idxes = concave_hull_indexes(bh_col_stack)

In [ ]:
bins = np.linspace(b_selected.min(), b_selected.max(), 100)
bin_ids = np.digitize(b_selected, bins)
bin_ids.shape

In [ ]:
min_h_per_bin = np.array([h_selected[bin_ids == i].min() for i in range(1, bins.size+1)])
max_h_per_bin = np.array([h_selected[bin_ids == i].max() for i in range(1, bins.size+1)])

In [ ]:
fig, ax = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(10, 5))
ax.plot(bins, max_h_per_bin, color='tab:orange')
ax.plot(bins, min_h_per_bin, color='tab:green')
ax.plot(bins, (max_h_per_bin + min_h_per_bin) / 2, color='tab:red')
ax.set_ylabel("H in A/m")
ax.set_xlabel("B in mT")
ax.set_title(f"{selected_temp} °C")
ax.grid(alpha=0.5)

In [ ]:
# dump to disk
pd.DataFrame({
    "bins_mT": bins,
    "min_H_A_per_m": min_h_per_bin,
    "max_H_A_per_m": max_h_per_bin,
    "avg_H_A_per_m": (min_h_per_bin + max_h_per_bin) / 2,
}).to_parquet(DATA_ROOT / "cache" / f"{mat_lbl}_1_25C_BH_bounds.parquet", index=False)